In [36]:
!pip install openpyxl
!pip install xlwings


In [1]:
import sys, pathlib
from pathlib import Path

# Setup paths
repo = pathlib.Path('.').resolve().parent.parent
sys.path.insert(0, str(repo))
sys.path.insert(0, str(repo / 'src'))


# in a notebook or Python script (after ensuring project is on sys.path or installed):
from instantgrade import InstantGrader
from pathlib import Path

# Point to the solution and submissions in the repo
solution = Path("./Assignment_ sol 1.xlsx")
subs = Path("./submissions")
report_dir = Path("./reports")
report_dir.mkdir(parents=True, exist_ok=True)

# Instantiate router and force Excel evaluator
grader = InstantGrader(
    solution_file_path=solution,
    submission_folder_path=subs,
    override_type="excel",        # forces Excel evaluator
    column_name="B",              # same defaults as the example notebook
    row_number=5,
    number_of_question=10,
)

report = grader.run()             # returns a ReportingService instance
html_path = report.to_html(report_dir / "excel_evaluation_report.html")
print("HTML report written to:", html_path)

HTML report written to: reports/excel_evaluation_report.html


/Volumes/MacSSD/Areas/Github_Repositories/evaluator/src/instantgrade/reporting/reporting_service.py:172: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["best_n_total"] = df["best_n_total"].fillna(0).astype(float)
/Volumes/MacSSD/Areas/Github_Repositories/evaluator/src/instantgrade/reporting/reporting_service.py:174: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["scaled"] = df["scaled"].fillna(0).astype(float)


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Checking Excel sheets

In [38]:
assignment_number = "Assignment_3"
assignment1_folder_path = "/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Test 3"
assignment2_excel_answer_key_path = "/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Assignments Files/Assignment_sol 3.xlsx"

In [39]:
import openpyxl
import os
import pandas as pd

def write_to_file(string_data, file_path=f"logfile_{assignment_number}.txt"):
  file_path = os.path.join(os.getcwd(), file_path)
  # print('write to file is executed', file_path)
  with open(file_path, 'a+') as file:
    file.writelines(string_data)


def get_all_excel_filenames(folder_path):
    files = os.listdir(folder_path)
    excel_files = []
    for f in files:
        if f.endswith('.xlsx'):
            excel_files.append(f)
    write_to_file(f"Found {len(excel_files)} excel files to evaluate. \n\n\n\n ")
    return excel_files


def check_value(cell_address, worksheet, log=True):
    if log:
        write_to_file(f" \n {worksheet[cell_address].value} \n ")
    return worksheet[cell_address].value


def check_data_type(cell_address, worksheet):
    write_to_file(f" \n {worksheet[cell_address].data_type} (__data_type) \n ")
    return worksheet[cell_address].data_type


def empty_data_structure():
    data_structure = dict()
    data_structure['Name'] = None
    data_structure['Roll Number'] = None
    data_structure['Result'] = 0
    data_structure['Error Sum'] = 0
    data_structure['Value Sum'] = 0
    data_structure['Formula Sum'] = 0

    return data_structure

try:
    import xlwings as xw
    xlwings_available = True
except ImportError:
    xlwings_available = False

def get_formula_multiengine(file_path, sheet_name, cell_address):
    # Try openpyxl first
    try:
        wb = openpyxl.load_workbook(file_path)
        ws = wb[sheet_name] if sheet_name in wb.sheetnames else wb.active
        cell = ws[cell_address]

        # Case 1: Cell has .formula attribute (rare)
        if hasattr(cell, "formula") and isinstance(cell.formula, str):
            return cell.formula, "openpyxl (cell.formula)"

        # Case 2: cell.value is a string starting with =
        if isinstance(cell.value, str) and cell.value.startswith("="):
            return cell.value, "openpyxl (cell.value)"

        # Case 3: cell.value is an ArrayFormula or object, reject and fall back
        if hasattr(cell.value, 'text'):
            return cell.value.text, "openpyxl (ArrayFormula.text)"

        # Case 4: Unusable object (e.g., openpyxl ArrayFormula), fall through to xlwings
    except Exception as e:
        # Continue to xlwings
        pass

    # Try xlwings if openpyxl didn't return usable formula
    if xlwings_available:
        try:
            app = xw.App(visible=False)
            wb = app.books.open(file_path)
            sht = wb.sheets[sheet_name] if sheet_name in [s.name for s in wb.sheets] else wb.sheets[0]
            formula = sht.range(cell_address).formula
            wb.close()
            app.quit()
            return formula, "xlwings"
        except Exception as e:
            return None, f"xlwings error: {str(e)}"

    return None, "No formula found"




def check_specific_answers(empty_data, column_name, row_number, number_of_question, formula_worksheet, value_worksheet,
                           values_answer_key, formula_answer_key, file_path):
    values_answer_key = values_answer_key or {}
    formula_answer_key = formula_answer_key or {}

    for i in range(number_of_question):
        write_to_file(f"{'-'* 4} \n \n Checking question number: {i+1}")

        cell_address = column_name + str(row_number + i)
        write_to_file(f"        checking cell:     [{cell_address}]  \n ")

        write_to_file(f"Checking formula output for any errors  \n ")
        value = check_value(cell_address, value_worksheet, False)
        is_error_value = value in (
            "#N/A", "#DIV/0!", "#NAME?", "#NULL!", "#NUM!", "#REF!", "#VALUE!", "#####", "Circular Reference"
        )

        if is_error_value:
            write_to_file(
                f"Error detected in value: >>{value}<<  \n "
            )
            empty_data['Error Sum'] += 1
            write_to_file(f"Your ERROR count increased: {empty_data['Error Sum']}  \n ")

        # Proceed with value checking anyway
        if values_answer_key is not None and cell_address in values_answer_key:
            if checking_if_values_are_correct(value, values_answer_key[cell_address]):
                empty_data['Value Sum'] += 1
                write_to_file(f"Your VALUE count increased: {empty_data['Value Sum']}  \n ")
            else:
                write_to_file(
                    f"   Warning: Output {value} of {type(value)} must be as specified in assignment : {values_answer_key[cell_address]}  of {type(values_answer_key[cell_address])}. \n "
                )
        else:
            write_to_file(f"          No output compulsion for this cell. \n ")

        # Proceed with formula checking regardless
        if formula_answer_key is not None and cell_address in formula_answer_key:
            formula_string, engine_used = get_formula_multiengine(file_path=file_path, sheet_name=formula_worksheet.title, cell_address=cell_address)

            try:
                if hasattr(formula_string, 'text'):
                    formula_string = formula_string.text
                elif not isinstance(formula_string, str):
                    formula_string = str(formula_string)
            except Exception as e:
                write_to_file(f"Error parsing formula: {e}\n")

            if not formula_string or not isinstance(formula_string, str):
                write_to_file("Could not retrieve usable formula from any engine.\n")
            else:
                write_to_file(f"Formula found using {engine_used}: {formula_string}\n")

                expected_results = formula_answer_key[cell_address]
                if isinstance(expected_results, str):
                    expected_results = [expected_results]

                formula_upper = formula_string.upper()

                if any(func.upper() in formula_upper for func in expected_results):
                    empty_data['Formula Sum'] += 1
                    write_to_file(f"Your FORMULA count increased: {empty_data['Formula Sum']}  \n ")
                else:
                    write_to_file(
                        f"   Warning: You should have used one of: {', '.join(expected_results)} but you used: {formula_string}.\n "
                    )
        else:
            write_to_file(f"          No formula compulsion for this cell. \n ")

        if cell_address in values_answer_key or cell_address in formula_answer_key:
            empty_data['Result'] += 1
            write_to_file(f"#####Your RESULT count increased: {empty_data['Result']} ###### \n ")


    return empty_data

def checking_if_values_are_correct(returned_value, actual_value):
    if returned_value == None or actual_value == None:
      return False

    if returned_value == actual_value:
        return True

    try:
        temp_returned_value = float(returned_value)
        temp_actual_value = float(actual_value)

        if temp_returned_value == temp_actual_value:
            return True
    except ValueError:
        pass

    if isinstance(actual_value, list):
      if returned_value in actual_value:
        return True

    if returned_value in ("True", "TRUE", "true", True) and actual_value in ("True", "TRUE", "true", True):
        return True

    elif returned_value in ("False", "FALSE", "false", False) and actual_value in ("False", "FALSE", "false", False):
        return True

    else:
        return False


def evaluate_excel_file(file_path, column_name, row_number, number_of_question, values_answer_key, formula_answer_key,
                        worksheet_name='Answers', student_name_cell='B2', student_roll_no_cell='B3'):
    write_to_file(f"{'~' * 80} \n\n \n ")

    if file_path:
        file_name = file_path.split('/')[-1]
        write_to_file(f" \n \n {'+'* 25} \n Working with {file_name}   \n \n")
    else:
        write_to_file("No file_path  \n ")
        return

    try:
        formula_wb = openpyxl.load_workbook(file_path)
    except:
        write_to_file(f"Could not find the {file_path}  \n ")
        return

    if worksheet_name not in formula_wb.sheetnames:
        new_worksheet = formula_wb.sheetnames[0]
        write_to_file(f"Since '{worksheet_name}' is not there. This tools is using {new_worksheet}  \n ")
        worksheet_name = new_worksheet
    try:
        formula_worksheet = formula_wb[worksheet_name]
    except KeyError:
        write_to_file("sheet named 'Answers' must be there and contain answers.  \n ")
        return

    try:
        value_wb = openpyxl.load_workbook(file_path, data_only=True)
    except:
        write_to_file(f"Could not find the {file_path}  \n ")
        return

    try:
        value_worksheet = value_wb[worksheet_name]
    except KeyError:
        write_to_file("sheet named 'Answers' must be there and contain answers.  \n ")
        return

    try:
        name = check_value(student_name_cell, formula_worksheet)
    except:
        write_to_file(f'Could not find your official Name in B1.  \n ')
        return

    try:
        roll_no = check_value(student_roll_no_cell, formula_worksheet)
    except:
        write_to_file(f'Could not find your roll number in B2.  \n ')
        return

    result_data = check_specific_answers(empty_data_structure(), column_name, row_number, number_of_question,
                                     formula_worksheet, value_worksheet, values_answer_key, formula_answer_key, file_path)

    result_data['Name'] = name
    result_data['Roll Number'] = roll_no

    write_to_file(str(result_data))
    write_to_file(f"{'*' * 80} \n\n \n ")

    return result_data


class CheckExcelFiles:
    def __init__(self, folder_path, formula_answer_key={}, values_answer_key={}):
        self.values_answer_key = values_answer_key
        self.path = folder_path
        self.formula_answer_key = formula_answer_key


    def evaluate_all_excel_files(self, column_name, row_number, number_of_question):
        result_data_list = []
        excel_filenames = get_all_excel_filenames(self.path)

        for filename in excel_filenames:

            file_path = os.path.join(self.path, filename)
            print(file_path)
            data = evaluate_excel_file(file_path, column_name, row_number, number_of_question, self.values_answer_key,
                                       self.formula_answer_key)
            print(data)
            if data:
                result_data_list.append(data)
        df = pd.DataFrame.from_records(result_data_list)

        return df



In [66]:
import re
from openpyxl.utils import column_index_from_string, get_column_letter

def extract_functions_from_formula(formula_string):
    if not isinstance(formula_string, str):
        return []
    return re.findall(r"\b[A-Z]+(?=\()", formula_string.upper())

def extract_nested_functions(formula: str, main_functions: list):
    """
    Extract functions nested inside the main function(s) only.
    Assumes formula starts with = and is a valid Excel formula.
    """
    if not isinstance(formula, str) or not formula.startswith("="):
        return []

    formula = formula[1:].strip()  # remove '='

    def parse_nested(func_str):
        stack = []
        tokens = []
        token = ""
        for char in func_str:
            if char == "(":
                if token:
                    stack.append(token.strip())
                    token = ""
                tokens.append("(")
            elif char == ")":
                if token:
                    tokens.append(token.strip())
                    token = ""
                tokens.append(")")
            elif char == ",":
                if token:
                    tokens.append(token.strip())
                    token = ""
                tokens.append(",")
            else:
                token += char
        if token:
            tokens.append(token.strip())
        return stack, tokens

    root_func = formula.split("(", 1)[0].strip().upper()

    # If the root function is not in the answer key, skip it
    if root_func not in [f.upper() for f in main_functions]:
        return []

    # Recursively extract functions inside parentheses
    nested_functions = set()
    stack = []
    buffer = ""
    for char in formula:
        if char == "(":
            if buffer:
                func_name = buffer.strip().upper()
                if func_name.isalpha():  # avoid numbers etc.
                    nested_functions.add(func_name)
                stack.append(func_name)
                buffer = ""
        elif char == ")":
            if buffer:
                buffer = ""
            if stack:
                stack.pop()
        else:
            buffer += char

    return list(nested_functions)

def create_formula_answers_loop(column_name, row_number, number_of_question, worksheet, nearby_columns=15):
    result = dict()
    start_col_idx = column_index_from_string(column_name)

    for i in range(number_of_question):
        row_idx = row_number + i
        main_cell = f"{column_name}{row_idx}"
        nested_functions = set()

        for offset in range(nearby_columns):
            col_idx = start_col_idx + offset
            col_letter = get_column_letter(col_idx)
            cell_address = f"{col_letter}{row_idx}"

            try:
                cell_value = check_value(cell_address, worksheet)
                if isinstance(cell_value, openpyxl.worksheet.formula.ArrayFormula):
                    cell_value = cell_value.text
                if isinstance(cell_value, str) and cell_value.startswith("="):
                    # Extract main function from the formula in the main cell
                    root_func = cell_value[1:].split("(", 1)[0].strip().upper()
                    nested = extract_nested_functions(cell_value, [root_func])
                    nested_functions.update(nested)
                else:
                    break  # stop if it's not a formula
            except Exception as e:
                write_to_file(f"Formula error at {cell_address}: {e}\n")
                break

        result[main_cell] = list(nested_functions)

    return result




def create_answers_loop(column_name, row_number, number_of_question, worksheet, nearby_columns=15):
    result = dict()
    start_col_idx = column_index_from_string(column_name)

    for i in range(number_of_question):
        row_idx = row_number + i
        main_cell = f"{column_name}{row_idx}"
        answer_values = set()

        for offset in range(nearby_columns):
            col_idx = start_col_idx + offset
            col_letter = get_column_letter(col_idx)
            cell_address = f"{col_letter}{row_idx}"

            try:
                cell_value = check_value(cell_address, worksheet)
                if cell_value is not None and str(cell_value).strip() != "":
                    answer_values.add(cell_value)
                else:
                    break  # stop at first empty cell
            except Exception as e:
                write_to_file(f"Value error at {cell_address}: {e}\n")
                break

        result[main_cell] = list(answer_values)

    return result

def create_answer_key(excel_file_path, column_name, row_number, number_of_question, worksheet_name=None, nearby_columns=15):
    # Load the formula workbook (with formulas)
    formula_wb = openpyxl.load_workbook(excel_file_path)
    if worksheet_name is None or worksheet_name not in formula_wb.sheetnames:
        worksheet_name = formula_wb.sheetnames[0]
        write_to_file(f"Using first sheet '{worksheet_name}' as answer sheet.\n")
    formula_ws = formula_wb[worksheet_name]

    # Load the value workbook (with evaluated values)
    value_wb = openpyxl.load_workbook(excel_file_path, data_only=True)
    if worksheet_name not in value_wb.sheetnames:
        worksheet_name = value_wb.sheetnames[0]
        write_to_file(f"Using first sheet '{worksheet_name}' from value workbook as answer sheet.\n")
    value_ws = value_wb[worksheet_name]

    # Create the answer keys
    formula_answer_key = create_formula_answers_loop(column_name, row_number, number_of_question, formula_ws, nearby_columns)
    value_answer_key = create_answers_loop(column_name, row_number, number_of_question, value_ws, nearby_columns)

    return formula_answer_key, value_answer_key


### **Instructions for Using the Tool**

**Step 1:** Provide the folder path where your solution Excel file is located.

**Step 2:** Mention the starting cell from where the answers begin in that Excel file. The evaluate_excel_file function tells which cell contain student_name_cell='B2', student_roll_no_cell='B3'

---

### **Example for Better Understanding**

* In the next input cell, enter the folder path where your solution Excel file is stored.
* The tool will look for answers starting from **cell B5** to **B14**, meaning a total of **10 questions**.
* Ensure that your answers are placed in **column B**, from row 5 to row 14.
* The tool will specifically check the sheet named **"Sheet1"** in your Excel file.

---


In [67]:
# creating the answer key from answer excel sheet

formula_answer_key, value_answer_key = create_answer_key(
    excel_file_path=assignment2_excel_answer_key_path,
    column_name='B',
    row_number=5,
    number_of_question=10,
    worksheet_name=None, nearby_columns=15
)

In [69]:
formula_answer_key

{'B5': ['FILTER', 'IFERROR'],
 'B6': ['MATCH', 'MAXIFS'],
 'B7': ['COUNTIFS'],
 'B8': ['SUMIFS'],
 'B9': ['AVERAGEIFS', 'IFERROR'],
 'B10': ['INDEX', 'MINIFS'],
 'B11': ['COUNTIFS'],
 'B12': ['XLOOKUP', 'MAXIFS'],
 'B13': ['SUMIFS'],
 'B14': ['INDEX', 'MAXIFS']}

In [68]:
formula_answer

{'B5': ['IFERROR',
  'MATCH',
  'COUNTIF',
  'IFERROR',
  'INDEX',
  'COUNTIFS',
  'XLOOKUP',
  'SUMIFS',
  'INDEX',
  'FILTER',
  'MAXIFS',
  'SUMIF',
  'AVERAGEIFS',
  'MINIFS',
  'VLOOKUP',
  'AVERAGEIF',
  'MAX',
  'AVERAGE',
  'PERCENTILE',
  'QUARTILE'],
 'B6': ['IFERROR',
  'MATCH',
  'COUNTIF',
  'IFERROR',
  'INDEX',
  'COUNTIFS',
  'XLOOKUP',
  'SUMIFS',
  'INDEX',
  'FILTER',
  'MAXIFS',
  'SUMIF',
  'AVERAGEIFS',
  'MINIFS',
  'VLOOKUP',
  'AVERAGEIF',
  'MAX',
  'AVERAGE',
  'PERCENTILE',
  'QUARTILE'],
 'B7': ['IFERROR',
  'MATCH',
  'COUNTIF',
  'IFERROR',
  'INDEX',
  'COUNTIFS',
  'XLOOKUP',
  'SUMIFS',
  'INDEX',
  'FILTER',
  'MAXIFS',
  'SUMIF',
  'AVERAGEIFS',
  'MINIFS',
  'VLOOKUP',
  'AVERAGEIF',
  'MAX',
  'AVERAGE',
  'PERCENTILE',
  'QUARTILE'],
 'B8': ['IFERROR',
  'MATCH',
  'COUNTIF',
  'IFERROR',
  'INDEX',
  'COUNTIFS',
  'XLOOKUP',
  'SUMIFS',
  'INDEX',
  'FILTER',
  'MAXIFS',
  'SUMIF',
  'AVERAGEIFS',
  'MINIFS',
  'VLOOKUP',
  'AVERAGEIF',
  'MAX',


In [59]:
value_answer

{'B5': 32,
 'B6': 467,
 'B7': 46,
 'B8': 116,
 'B9': 2318.987854,
 'B10': 'Austin',
 'B11': 87,
 'B12': 'MODEL S',
 'B13': 350750,
 'B14': 'Battery Electric Vehicle (BEV)'}

In [60]:
values = ['IFERROR',
  'MATCH',
  'COUNTIF',
  'IFERROR',
  'INDEX',
  'COUNTIFS',
  'XLOOKUP',
  'SUMIFS',
  'INDEX', 'FILTER', 'MAXIFS', 'SUMIF', 'AVERAGEIFS', 'MINIFS', 'VLOOKUP', 'AVERAGEIF', 'MAX', 'AVERAGE', 'PERCENTILE', 'QUARTILE', ]

formula_answer = {
    key: list(values)
    for key in formula_answer
}


In [61]:
custom_formula_answer = {'B5': '=IFERROR',
 'B6': '=MATCH',
 'B7': '=COUNTIFS',
 'B8': '=SUMIFS',
 'B9': '=IFERROR',
 'B10': '=INDEX',
 'B11': '=COUNTIFS',
 'B12': '=XLOOKUP',
 'B13': '=SUMIFS',
 'B14': '=INDEX'}

In [62]:
custom_value_answer = {'B5': 32,
 'B6': 467,
 'B7': 46,
 'B8': 116,
 'B9': 2318.987854,
 'B10': 'Austin',
 'B11': 87,
 'B12': 'MODEL S',
 'B13': 350750,
 'B14': 'Battery Electric Vehicle (BEV)'}

# Step 3
You have to give the path of folder which consist students excel files. It will check from B5 till 10 questions for each students.

In [63]:

checker_object = CheckExcelFiles(assignment1_folder_path, formula_answer, value_answer)

result = checker_object.evaluate_all_excel_files('B',  5, 10)


/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Test 3/Assignment_3 name roll no (17).xlsx
{'Name': 'Prapti Das', 'Roll Number': 2423142, 'Result': 10, 'Error Sum': 1, 'Value Sum': 7, 'Formula Sum': 10}
/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Test 3/Assignment_3 name roll no (1) (4).xlsx
{'Name': None, 'Roll Number': None, 'Result': 10, 'Error Sum': 1, 'Value Sum': 8, 'Formula Sum': 10}
/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Test 3/Assignment_3_Kabir 2423123.xlsx
{'Name': 'Kabir Sengupta', 'Roll Number': 2423123, 'Result': 10, 'Error Sum': 0, 'Value Sum': 9, 'Formula Sum': 10}
/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Test 3/Asmee.xlsx
{'Name': None, 'Roll Number': None, 'Result': 10, 'Error Sum': 0, 'Value Sum': 3, 'Formula Sum': 1}
/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BB

In [64]:
result['Name'] = result['Name'].apply(lambda x: str(x).strip().title())

In [65]:
new_df = result.sort_values(by='Formula Sum', ascending= False)

# new_df.to_csv("Assignment_1 Metrics", index = False)
clean_df = new_df.drop_duplicates(subset=['Roll Number'])
clean_df.to_csv(f"{assignment_number} Metrics.csv", index = False)

clean_df

,Name,Roll Number,Result,Error Sum,Value Sum,Formula Sum
0,Prapti Das,2423142.0,10,1,7,10
1,None,NaN,10,1,8,10
2,Kabir Sengupta,2423123.0,10,0,9,10
4,Manya Lakhmani,2423136.0,10,1,7,10
6,Sanika Khoche,2423149.0,10,0,5,10
12,Ankisha Jain,2423111.0,10,0,5,10
10,Kavyasree G,2423126.0,10,0,7,10
14,Sakshi Venugopal,2423147.0,10,1,7,10
13,Noor Aminah,2423140.0,10,1,8,10
16,Akshay Shah,2423106.0,10,1,7,10


# EOF